In [ ]:
#Importing modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io.votable import parse
from astropy.io.votable import parse_single_table
from astropy import constants as const
import csv
import requests

In [ ]:
votable_file_path = 'vizier_votable.vot'
#Parsing the VOTable data file
votable = parse_single_table(votable_file_path)
#Convert data to pandas data fame
data=votable.to_table().to_pandas()

In [ ]:
# List of inaccurate catalog names to be removed
catalogs_to_exclude = ['SkyMapper', 'XMM', 'GAIA', 'Gaia:G', 'VISTA','GAIA/GAIA2:Grp','GAIA/GAIA2:G','GAIA/GAIA2:Gbp','GAIA/GAIA3:Grp','GAIA/GAIA3:G','GAIA/GAIA3:Gbp','VISTA:Ks','VISTA:Y','VISTA:J','XMM-OT:UVW1','SkyMapper/SkyMapper:z','SkyMapper/SkyMapper:i','SkyMapper/SkyMapper:r','SkyMapper/SkyMapper:g','SkyMapper/SkyMapper:v','SkyMapper/SkyMapper:u']

# Use drop to remove rows with inaccurate catalog names
reduced_data = data.drop(data[data['sed_filter'].isin(catalogs_to_exclude)].index)

# Display the updated DataFrame
print(len(reduced_data))


211


In [ ]:
# Create a copy of the original DataFrame to preserve the reduced data
data_1 = reduced_data.copy() # Contains data points that have error ratio greater than 0.5

for i in data_1.index:
    ratio = data_1.loc[i, 'sed_eflux'] / data_1.loc[i, 'sed_flux']
    if ratio > 0.05:
        data_1.drop(i, inplace=True)

In [ ]:
for i in data_1.index:
    if data_1.loc[i, 'sed_eflux'] == 0:
        data_1.drop(i, inplace=True)

In [ ]:
#Save data_1 as csv
final_data = pd.DataFrame(data_1)

# Save the DataFrame to a CSV file
final_data.to_csv('final_observed_data.csv', index=False)

In [ ]:
#Model Data

url = "https://wwwuser.oats.inaf.it/fiorella.castelli/grids/gridm10k2odfnew/fm10t8250g10k2odfnew.dat"
output_csv_file = "model_data.csv"

kurucz_data = pd.read_csv(url, skiprows=1, delim_whitespace=True)

kurucz_data.rename(columns={'TITLE': 'x','[-1.0]': 'y','VTURB=2': 'model_wavelength[nm]','L/H=1.25': 'frequency nu [s^-1]',
    'NOVER': 'Hnu (erg/cm^2/s/Hz/ster)','NEW': 'Hcont (erg/cm^2/s/Hz/ster)','ODF': 'Hnu/Hcont'}, inplace=True)


kurucz_data.to_csv(output_csv_file, index=False)

In [ ]:
dat_file = 'Rv_3_1.dat'
# Read the data from the .dat file
red_data = pd.read_csv(dat_file, skiprows=3, delim_whitespace=True)
red_data['red_wv']=red_data['Angstrom']*0.1
red_data.to_csv('red_data.csv', index=False)

In [ ]:
#calculating wavelength in nm of observed data
c=const.c
sed_observed_frequency_ghz=final_data["sed_freq"]
final_data['sed_observed_wavelength_nm'] = c / sed_observed_frequency_ghz  # in nm
final_data['flux_sed_observed'] = final_data["sed_flux"]*(10**(-23)) #erg/s/cm²/Hz
final_data['eflux_sed_observed'] = final_data["sed_eflux"]*(10**(-23)) #erg/s/cm²/Hz
final_data['sed_observed_frequency_hz']=final_data["sed_freq"]*(10**9) #in Hz
final_data.to_csv('final_observed_data.csv', index=False)